# HW1 布尔查询之BSBI与索引压缩

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [1]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib
from collections import defaultdict

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [2]:
# import urllib.request
# import zipfile

# data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
# data_dir = 'pa1-data'
# urllib.request.urlretrieve(data_url, data_dir+'.zip')
# zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
# zip_ref.extractall()
# zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [3]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [4]:
sorted(os.listdir("pa1-data"))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [5]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [6]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [7]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [8]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.str_to_id:
            sid = len(self.id_to_str)
            self.str_to_id[s] = sid
            self.id_to_str.append(s)
        else:
            sid = self.str_to_id[s]
        return sid
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        # else:
        #     raise TypeError

确保代码能通过以下简单测试样例

In [9]:
testIdMap = IdMap()


assert testIdMap["a"] == 0, "Unable to add a new string to the IdMap"
assert testIdMap["bcd"] == 1, "Unable to add a new string to the IdMap"
assert testIdMap["a"] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

在这一步发现如果字符串使用单引号返回type会是str，而双引号是**<class ,str>**，中途调试出现错误，因此将源代码中的字符串改成了双引号

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [10]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """
        将倒排列表编码成字节流
        
        参数
        ----------
        postings_list: List[int]
            文档ID列表（倒排列表）
            
        返回
        -------
        bytes
            表示倒排列表中整数的字节数组
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """
        从字节流解码倒排列表
        
        参数
        ----------
        encoded_postings_list: bytes
            表示编码后倒排列表的字节数组
            
        返回
        -------
        List[int]
            从编码后的字节数组解码出的文档ID列表
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()


运行以下代码查看其工作方式

In [11]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [12]:
class InvertedIndex:
    """一个实现高效读写倒排索引到磁盘的类
    
    属性
    ----------
    postings_dict: 字典映射: termID->(索引文件中的起始位置, 
                                      列表中的记录数,
                                      列表的字节长度)
        这是一个字典，从termID映射到一个包含元数据的三元组，
        这些元数据有助于在磁盘上读写索引文件中的记录。
        这个映射应该保存在内存中。
        索引文件中的起始位置是记录列表在索引文件中的字节位置。
        列表中的记录数是记录列表中的记录（docID）的数量。
        列表的字节长度是记录列表的字节编码长度。
    
    terms: List[int]
        一个termID的列表，用于记住将术语及其记录列表添加到索引中的顺序。
        
        在Python 3.7之后，我们实际上不再需要它，因为Python字典已经是有序字典，
        但由于这是一个相对较新的特性，我们仍然保持向后兼容性，
        使用列表来跟踪插入顺序。
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        参数
        ----------
        index_name (str): 用于存储与索引相关的文件的名称
        postings_encoding: 一个实现编码和解码整数列表的静态方法的类。
            默认为None，将被替换为UncompressedPostings
        directory (str): 存储索引文件的目录
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []          #需要跟踪插入术语的顺序。从Python 3.7开始不再需要

    def __enter__(self):
        """进入上下文时打开索引文件并加载元数据"""
        # 打开索引文件
        self.index_file = open(self.index_file_path, 'rb+')

        # 从元数据文件加载postings_dict和terms
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """退出上下文时关闭索引文件并保存元数据"""
        # 关闭索引文件
        self.index_file.close()
        
        # 将postings_dict和terms写入元数据文件
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [13]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    属性
    ----------
    term_id_map(IdMap): 用于将术语映射到术语ID
    doc_id_map(IdMap): 用于将文档的相对路径（例如 0/3dradiology.stanford.edu_）映射到文档ID
    data_dir(str): 数据路径
    output_dir(str): 输出索引文件的路径
    index_name(str): 分配给索引的名称
    postings_encoding: 用于存储倒排列表的编码。
        默认值（None）表示未压缩的倒排列表
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # 存储中间索引的名称
        self.intermediate_indices = []
        
    def save(self):
        """将 doc_id_map 和 term_id_map 转储到输出目录"""
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """从输出目录加载 doc_id_map 和 term_id_map"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """基本索引代码
        
        该函数遍历数据目录，
        调用 parse_block 解析文档
        调用 invert_write 反转每个块并写入新索引
        然后保存 id 映射并调用 merge 合并中间索引
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [14]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """解析一个标记化的文本文件为 termID-docID 对

        参数
        ----------
        block_dir_relative : str
            包含该块文件的目录的相对路径
        
        返回
        -------
        List[Tuple[Int, Int]]
            返回从该块中提取的所有 td_pairs
        
        应使用 self.term_id_map 和 self.doc_id_map 获取 termID 和 docID。
        这些在对 parse_block 的调用之间保持不变。
        """
        ### Begin your code
        td_pairs = []
        block_dir = os.path.join(self.data_dir,block_dir_relative)
        for doc_name in os.listdir(block_dir):
            doc_path = os.path.join(block_dir,doc_name)
            doc_id = self.doc_id_map[os.path.join(block_dir_relative,doc_name)]
            with open(doc_path,'r') as f:
                terms = f.read().split()
                for term in terms:
                    term_id = self.term_id_map[term]
                    td_pairs.append((term_id,doc_id))
        return td_pairs
        ### End your code

在这一步，逐个遍历指定块中的所有文件，为其分配文件id，并将其中的文本拆分成词项列表进行遍历，从而得到termID-DocID对

观察函数在测试数据上是否正常运行

In [15]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [16]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (4, 1),
 (8, 1)]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [17]:
### Begin your code

### End your code

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [18]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """将术语和文档ID列表追加到索引文件末尾。
        
        这个函数做三件事：
        1. 使用 self.postings_encoding 对文档ID列表进行编码
        2. 以 self.terms 和 self.postings_dict 的形式存储元数据
           注意，self.postings_dict 将 termID 映射到一个包含以下三项的元组：
           (索引文件中的起始位置, 
           文档ID列表中的文档数量, 
           文档ID列表的字节长度)
        3. 将字节流追加到磁盘上的索引文件末尾

        提示：你可能会发现阅读 Python I/O 文档（https://docs.python.org/3/tutorial/inputoutput.html）关于文件末尾追加的部分很有帮助。
        
        参数
        ----------
        term:
            术语或 termID 是术语的唯一标识符
        postings_list: List[Int]
            包含术语出现的文档ID的列表
        """
        ### Begin your code
        doc_id_code = self.postings_encoding.encode(postings_list)
        startpoint = self.index_file.tell() # 当前字节偏移地址（相当于定位到光标地址）
        self.index_file.write(doc_id_code) # 进行写入
        self.terms.append(term)
        self.postings_dict[term] = (startpoint ,len(postings_list),len(doc_id_code)) # 存储元数据
        self.index_file.flush()
        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [19]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [20]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """将 td_pairs 转换为 postings_lists 并写入给定的索引
        参数
        ----------
        td_pairs: List[Tuple[Int, Int]]
            词项ID-文档ID对的列表
        index: InvertedIndexWriter
            磁盘上的倒排索引，对应于块
        """
        ### Begin your code
        postings_lists = defaultdict(list)
        for term_id, doc_id in td_pairs:
            if doc_id not in postings_lists[term_id]:
                postings_lists[term_id].append(doc_id)
        for term_id in sorted(postings_lists.keys()): # 在这一步按顺序添加倒排索引，方便后续合并
            index.append(term_id, postings_lists[term_id])
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [21]:
### Begin your code
with InvertedIndexWriter('test', directory='tmp/') as index:
    bsbi = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
    td_pairs = bsbi.parse_block('0')
    bsbi.invert_write(td_pairs,index)
with open('tmp/test.index','rb') as ind:
    print("这是解码后的文件：",str(UncompressedPostings.decode(ind.read())))
with open('tmp/test.dict','rb') as dic:
    postings_dict, terms = pkl.load(dic)
    print("Postings Dictionary:", postings_dict)
    print("Terms:", terms)
### End your code

这是解码后的文件： [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Postings Dictionary: {0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 1, 4), 3: (12, 1, 4), 4: (16, 2, 8), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4), 8: (36, 1, 4)}
Terms: [0, 1, 2, 3, 4, 5, 6, 7, 8]


### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [22]:
class InvertedIndexIterator(InvertedIndex):
    def __enter__(self):
        """在进入上下文时添加初始化钩子"""
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """使用此函数初始化迭代器"""
        ### Begin your code
        self.current = 0
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """返回索引中的下一个 (term, postings_list) 对。
        
        注意：此函数应仅从索引文件中读取少量数据。
        特别是，你不应该尝试在内存中维护完整的索引文件。
        """
        ### Begin your code
        if self.current >= len(self.terms):
            raise StopIteration
        
        term = self.terms[self.current]
        start_position,doc_num,listsize = self.postings_dict[term]
        self.index_file.seek(start_position)
        posting_list = self.postings_encoding.decode(self.index_file.read(listsize))
        self.current += 1

        return (term, posting_list)
        ### End your code

    def delete_from_disk(self):
        """标记索引在退出时删除。对于临时索引很有用"""
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """在退出上下文时删除索引文件以及父类的 __exit__ 函数"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [23]:
### Begin your code
with InvertedIndexWriter('test', directory='tmp/') as index:
    bsbi = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
    td_pairs = bsbi.parse_block('0')
    bsbi.invert_write(td_pairs,index)

with InvertedIndexIterator('test',directory='tmp/') as read:
    for term,posting_list in read:
        print(f"Term: {term}, Postings List: {posting_list}")
    read.delete_from_disk
        
### End your code

Term: 0, Postings List: [0]
Term: 1, Postings List: [0]
Term: 2, Postings List: [0]
Term: 3, Postings List: [0]
Term: 4, Postings List: [0, 1]
Term: 5, Postings List: [1]
Term: 6, Postings List: [1]
Term: 7, Postings List: [1]
Term: 8, Postings List: [1]


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [24]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [25]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """将多个倒排索引合并为一个单一索引

        参数
        ----------
        indices: List[InvertedIndexIterator]
            一个InvertedIndexIterator对象的列表，每个对象代表一个块的可迭代倒排索引
        merged_index: InvertedIndexWriter
            一个InvertedIndexWriter对象的实例，每次将合并后的记录列表写入其中
        """
        ## Begin your code
        now_term = -1
        now_postings = None
        for next_term,next_posting in heapq.merge(*indices,key=lambda x: x[0]):
            if now_term == next_term:
                now_postings.extend(next_posting)
            else :
                if now_term!= -1:
                    merged_index.append(now_term,now_postings)
                now_postings = next_posting
                now_term = next_term
        if now_term!=-1:
            merged_index.append(now_term,now_postings)
        ### End your code

首先确保它在测试数据上正常运行

In [27]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

print(BSBI_instance.term_id_map.str_to_id)
print(BSBI_instance.doc_id_map.str_to_id)
with InvertedIndexIterator('BSBI',directory='toy_output_dir/') as read:
    for term,posting_list in read:
        print(f"Term: {term}, Postings List: {posting_list}")
    read.delete_from_disk
    

{"i'm": 0, 'fine': 1, ',': 2, 'thank': 3, 'you': 4, 'hi': 5, 'how': 6, 'are': 7, '?': 8, 'good': 9, 'to': 10, 'see': 11, 'bye': 12, 'later': 13}
{'0\\fine.txt': 0, '0\\hello.txt': 1, '1\\bye.txt': 2, '1\\byebye.txt': 3, '2\\fine.txt': 4, '2\\hello.txt': 5}
Term: 0, Postings List: [0, 4]
Term: 1, Postings List: [0, 4]
Term: 2, Postings List: [0, 2, 4]
Term: 3, Postings List: [0, 4]
Term: 4, Postings List: [0, 1, 2, 4, 5]
Term: 5, Postings List: [1, 5]
Term: 6, Postings List: [1, 5]
Term: 7, Postings List: [1, 5]
Term: 8, Postings List: [1, 2, 5]
Term: 9, Postings List: [2]
Term: 10, Postings List: [2]
Term: 11, Postings List: [2]
Term: 12, Postings List: [2, 3]
Term: 13, Postings List: [2]


In [36]:
# BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
# BSBI_instance.index()

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [28]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [29]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """
        获取 `term` 的文档ID列表（postings list）。

        这个函数不应该遍历索引文件。
        也就是说，它只需要读取索引文件中对应于请求的术语的文档ID列表的字节。
        """
        ### Begin your code
        if term < len(self.terms):
            start_point ,len_doc,len_in_bytes = self.postings_dict[term]
            self.index_file.seek(start_point)
            posting_list_in_byte = self.index_file.read(len_in_bytes)
            ret = self.postings_encoding.decode(posting_list_in_byte)
            return ret
        else:
            return []
        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [30]:
### Begin your code
with InvertedIndexMapper('BSBI',directory='toy_output_dir/') as map:
    map_list = map._get_postings_list(5)
    print(map_list)
### End your code

[1, 5]


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [31]:
def sorted_intersect(list1, list2):
    """
    交集两个（升序）排序的列表并返回排序结果

    参数
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        要交集的排序列表

    返回
    -------
    List[Comparable]
        排序后的交集
    """
    ### Begin your code
    i1 = 0
    i2 = 0
    list = []
    while ((i1<len(list1))&(i2<len(list2))):
        if list1[i1] < list2[i2]:
            i1 += 1
        else:
            if list2[i2] < list1[i1]:
                i2 += 1
            else:
                list.append(list1[i1])
                i1 += 1
                i2 += 1
    return list
    ### End your code

简单的测试样例

In [32]:
### Begin your code
list1= [1,2,3,5,6,7]
list2=[2,3,4,5,6]
print(sorted_intersect(list1,list2))
### End your code

[2, 3, 5, 6]


现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [33]:
# %%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """
        检索与合取查询对应的文档

        参数
        ----------
        query: str
            空格分隔的查询词列表

        返回
        ------
        List[str]
            包含每个查询词的文档的排序列表。
            如果没有找到文档，则应为空。

        不应对语料库中不存在的术语抛出错误
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()
        ### Begin your code
        with InvertedIndexMapper(self.index_name,postings_encoding=self.postings_encoding,directory=self.output_dir) as map:
            word_list = query.split() # 分词
            ret = None
            for word in word_list:
                term = self.term_id_map[word] # 获取词项id
                if not ret:
                    ret = map._get_postings_list(term) # 包含该词项的文件列表
                else:
                    ret = sorted_intersect(ret,map[term]) # 进行合并
            return [self.doc_id_map[retid] for retid in ret]
        ### End your code

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [43]:
BSBI_instance = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir', )
BSBI_instance.retrieve('hi')

['0\\hello.txt', '2\\hello.txt']

In [34]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('we are')

['0\\5-sure.stanford.edu_',
 '0\\50years.stanford.edu_',
 '0\\a3cservices.stanford.edu_lead_',
 '0\\aa.stanford.edu_',
 '0\\aa.stanford.edu_about_control.php',
 '0\\aa.stanford.edu_admissions_',
 '0\\aa.stanford.edu_admissions_admittedGrad.php',
 '0\\aa.stanford.edu_admissions_fellowsAero.php',
 '0\\aa.stanford.edu_events_50thAnniversary_media_Ying.pdf',
 '0\\aa.stanford.edu_faculty_facultySearch.php',
 '0\\aa.stanford.edu_giving_index.php',
 '0\\aaalab.stanford.edu_child_development_dev_dominos.html',
 '0\\aaalab.stanford.edu_child_development_dev_reasoning.html',
 '0\\aaalab.stanford.edu_face_to_face_interaction_face_agency.html',
 '0\\aaalab.stanford.edu_imagery_im_complex.html',
 '0\\aaalab.stanford.edu_teachable_agents_ta_moby.html',
 '0\\aaastop.stanford.edu_patients_',
 '0\\aaastop.stanford.edu_patients_our_goal.html',
 '0\\aaastop.stanford.edu_patients_who_is_eligible.html',
 '0\\aaastop.stanford.edu_physicians_',
 '0\\aacf.stanford.edu_',
 '0\\aagsa.stanford.edu_content_grad-f

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [35]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [36]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [37]:
# (set(my_results) - set(reference_results))

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [38]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [39]:
import array

class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def vb_encode_number(number):
        # 使用可变字节编码对单个数字进行编码
        bytes_list = []
        while True:
            bytes_list.insert(0, number % 128)  # 将字节前置到列表中
            if number < 128: # 数字的第一个字节
                break
            number //= 128
        bytes_list[-1] += 128  # 在最后一个字节中设置继续位
        return bytes_list

    @staticmethod
    def vb_encode(numbers):
        # 使用可变字节编码对数字列表进行编码
        bytes_list = []
        for number in numbers:
            bytes_list.extend(CompressedPostings.vb_encode_number(number))
        return bytes_list

    @staticmethod
    def vb_decode(bytes_list):
        # 使用可变字节编码对字节列表进行解码
        numbers = []
        number = 0
        # 逐个字节的进行解码
        for byte in bytes_list:
            if byte < 128:# 如果当前的字节表示的数字小于128（当前数字还有其他字节）
                number = 128 * number + byte
            else: # 否则就去掉第一位的标志位，加到之前的结果中就是当前的数字
                number = 128 * number + (byte - 128)
                numbers.append(number)
                number = 0
        return numbers

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """
        使用间隔编码和可变字节编码对 `postings_list` 进行编码
        
        参数
        ----------
        postings_list: List[int]
            要编码的倒排记录表
        
        返回
        -------
        bytes: 
            压缩后的倒排记录表的字节表示
            （由 `array.tobytes` 函数生成）
        """
        ### Begin your code
        if not postings_list:
            return b''
        gaps = [postings_list[0]] + [postings_list[i] - postings_list[i - 1] for i in range(1, len(postings_list))]
        encoded_bytes = CompressedPostings.vb_encode(gaps)
        return array.array('B', encoded_bytes).tobytes()
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """
        对压缩后的倒排记录表的字节表示进行解码
        
        参数
        ----------
        encoded_postings_list: bytes
            由 `CompressedPostings.encode` 生成的字节表示
            
        返回
        -------
        List[int]
            解码后的倒排记录表（每个记录是一个文档ID）
        """
        ### Begin your code
        if not encoded_postings_list:
            return []
        gaps = CompressedPostings.vb_decode(list(encoded_postings_list))
        postings_list = [gaps[0]]
        for i in range(1, len(gaps)):
            postings_list.append(postings_list[-1] + gaps[i])
        return postings_list
        ### End your code


首先，如果你实现了额外的辅助函数，写一些测试样例

In [40]:
### Begin your code
assert CompressedPostings.vb_encode_number(128) == [1, 128]
assert CompressedPostings.vb_encode_number(127) == [255]
assert CompressedPostings.vb_encode_number(0) == [128]
assert CompressedPostings.vb_encode([128, 127, 0]) == [1, 128, 255, 128]
assert CompressedPostings.vb_decode([1, 128, 255, 128]) == [128, 127, 0]
postings_list = [1, 2, 3, 10, 20]
encoded = CompressedPostings.encode(postings_list)
decoded = CompressedPostings.decode(encoded)
assert decoded == postings_list
print("All tests passed!")
### End your code

All tests passed!


我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [41]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [42]:
### Begin your code

### End your code

现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [43]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [44]:
# BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
# BSBI_instance_compressed.index()

In [45]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

像之前一样，用已构造好的查询语句来测试是否正常运行

In [47]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [48]:
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def gamma_encode_number(number):
        # 使用Gamma编码对单个数字进行编码
        # 为了方便处理0的问题，我们编码前将数字加一
        number += 1
        binary = bin(number)[2:]  # 获取二进制表示，去掉 '0b' 前缀
        offset = binary[1:]  # 去掉最高位的1
        length = len(offset)
        unary = '0' * length + '1'
        return unary + offset

    @staticmethod
    def gamma_encode(numbers):
        # 使用Gamma编码对数字列表进行编码
        return ''.join(ECCompressedPostings.gamma_encode_number(number) for number in numbers)

    @staticmethod
    def gamma_decode(encoded_str):
        # 使用Gamma编码对字符串进行解码
        numbers = []
        i = 0
        while i < len(encoded_str):
            length = 0
            while encoded_str[i] == '0':
                length += 1
                i += 1
            i += 1  # 跳过 '0'
            if length == 0:
                numbers.append(0)
            else:
                offset = encoded_str[i:i + length]
                number = int('1' + offset, 2)
                numbers.append(number - 1)
                i += length
        return numbers

    @staticmethod
    def to_bytes(bitstring):
        # 将二进制字符串转换为字节，并在最后添加补零的位数
        padding_length = (8 - len(bitstring) % 8) % 8 # 计算补零的位数
        bitstring = bitstring + '0' * padding_length
        b = bytearray()
        for i in range(0, len(bitstring), 8):
            byte = bitstring[i:i + 8]
            b.append(int(byte, 2))
        # 在最后添加补零的位数
        b.append(padding_length)
        return bytes(b)

    @staticmethod
    def from_bytes(byte_data):
        # 将字节转换为二进制字符串，并根据最后的位数去掉补零
        padding_length = byte_data[-1]# 获取补零的位数
        bitstring = ''.join(f'{byte:08b}' for byte in byte_data[:-1])
        return bitstring[:-padding_length] if padding_length > 0 else bitstring
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """对 `postings_list` 进行编码
        
        参数
        ----------
        postings_list: List[int]
            要编码的倒排列表
        
        返回
        -------
        bytes: 
            压缩后的倒排列表的字节表示
        """
        ### Begin your code
        if not postings_list:
            return b''
        gaps = [postings_list[0]] + [postings_list[i] - postings_list[i - 1] for i in range(1, len(postings_list))]
        encoded_str = ECCompressedPostings.gamma_encode(gaps)
        return ECCompressedPostings.to_bytes(encoded_str)
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """对压缩后的倒排列表的字节表示进行解码
        
        参数
        ----------
        encoded_postings_list: bytes
            由 `CompressedPostings.encode` 生成的字节表示
            
        返回
        -------
        List[int]
            解码后的倒排列表（每个倒排项是一个文档ID）
        """
        ### Begin your code
        if not encoded_postings_list:
            return []
        encoded_str = ECCompressedPostings.from_bytes(encoded_postings_list)
        gaps = ECCompressedPostings.gamma_decode(encoded_str)
        postings_list = [gaps[0]]
        for i in range(1, len(gaps)):
            postings_list.append(postings_list[-1] + gaps[i])
        return postings_list
        ### End your code


写一些测试代码

In [49]:
# 测试代码
# 测试 gamma_encode_number 和 gamma_decode
number = 0
encoded_number = ECCompressedPostings.gamma_encode_number(number)
decoded_number = ECCompressedPostings.gamma_decode(encoded_number)
assert decoded_number == [number], f"gamma_decode({encoded_number}) != {number}"

# 测试 gamma_encode 和 gamma_decode
numbers = [0,1,2,3,34,40,46,67,87,100,109,150,198,400,567,890,1278]
encoded_numbers = ECCompressedPostings.gamma_encode(numbers)
decoded_numbers = ECCompressedPostings.gamma_decode(encoded_numbers)
assert decoded_numbers == numbers, f"gamma_decode({encoded_numbers}) != {numbers}"

# 测试 to_bytes 和 from_bytes
bitstring = '11001010111110100001000100010010'
byte_data = ECCompressedPostings.to_bytes(bitstring)
decoded_bitstring = ECCompressedPostings.from_bytes(byte_data)
assert decoded_bitstring[:len(bitstring)] == bitstring, f"from_bytes({byte_data}) != {bitstring}"

# 测试 encode 和 decode
postings_list = [1,2,3,34,40,46,67,87,100,109,150,198,400,567,890,1278]
encoded = ECCompressedPostings.encode(postings_list)
decoded = ECCompressedPostings.decode(encoded)
assert postings_list == decoded, f"decode({encoded}) != {postings_list}"

print("所有测试通过!")


所有测试通过!


现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [50]:
try: 
    os.mkdir('output_dir_ECCcompressed')
except FileExistsError:
    pass

In [51]:
BSBI_instance_ECCcompressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_ECCcompressed', postings_encoding=ECCompressedPostings)
BSBI_instance_ECCcompressed.index()

In [53]:
BSBI_instance_ECCcompressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_ECCcompressed', postings_encoding=ECCompressedPostings)
Str = 'you'
set(BSBI_instance_ECCcompressed.retrieve(Str))-set(BSBI_instance_compressed.retrieve(Str))

set()

In [54]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


In [55]:
(set(my_results) - set(reference_results))

set()

In [56]:
import os
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size
uncompressed_size = get_folder_size('output_dir')
compressed_size = get_folder_size('output_dir_compressed')
ec_compressed_size = get_folder_size('output_dir_ECCcompressed')

print(f"Uncompressed index size: {uncompressed_size} bytes")
print(f"Compressed index size: {compressed_size} bytes")
print(f"EC Compressed index size: {ec_compressed_size} bytes")

Uncompressed index size: 145802835 bytes
Compressed index size: 68253760 bytes
EC Compressed index size: 63253913 bytes


# 作业提交

本次作业用时y约3周，截止日期为9.30（国庆节前）。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到ir24fall@163.com，邮件和文件命名方式均为`学号_姓名_hw1`，如`1811412_戚晓睿_hw1.ipynb`

实验步骤已经在前面写的比较详细，在此处记录下完成本次作业的一些心得体会

1. 第一次比较完成的实现了一个功能非常使用的查询系统，虽然比较初级，但是对于文件的读写，对于倒排列表的增删合并等操作增进了许多了解
2. 在倒排列表的建立过程中，我在最初的代码中常常使用了诸如 **in**，**set()**，**sort()**等函数甚至是嵌套使用，导致整体的复杂度非常高，建立索引的时间长到爆炸，在之后请教助教，更加熟悉了整个系统，另一方面也注意到了时间复杂度的问题，经过非常之久的排查后成功将建立索引的时间控制在15分钟以内，这样的经验和体会更让我意识到复杂度的问题
3. 在对索引进行变长字节编码和gap编码的过程中，由于经常出错，导致我不得不在函数的每一关键步骤进行输出调试，观察每一步的问题，并且让我对python更加熟悉
4. 在尝试使用gamma编码的时候，我崩溃的发现我的结果和答案差很多，仔细排查依然没有发现问题所在，不得不一遍遍的读代码去找bug，才发现gamma编码没有对于0的编码，于是我构想使用单个'1'来对0进行编码，但是尝试后发现，这样子编码就会产生歧义，导致解码非常之离谱。最后经过缜密的思考，我想到可以在进行编码之前将列表的每个元素+1，然后解码时候再-1，这样子就避免了列表中出现0导致编码失败，这样的过程让我认识到，编码的一大特性就是没有歧义，否则就会产生许多问题
5. 在前一个部分进行调试的时候，需要经常对之前的依赖模块进行运行，导致最后我的整个文件运行顺序混乱不堪，并且经常误触运行了索引产生模块，于是我将一些耗时很长且不用必须运行的模块进行注释，每次需要的时候就将整个文件从头运行。
6. 这次的作业框架，将一个布尔查询的项目拆解成许多个步骤，引导我一步步实现一些小模块，最后通过之前的小模块实现一个较大的项目，完整的实现这样的过程让我体悟到许多编程思想。